# 1. ollama 서버 상태 확인
#### 서버 실행되지 않았을 때, ConnectionError 방지

In [ ]:
import requests

SERV_URL = "http://127.0.0.1:11434"
# 서버 연결 체크 함수 
def check_server_status(base_url="http://localhost:11434"):
    try:
        response = requests.get(base_url)
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        return False

# 서버가 켜져 있는지 확인
if not check_server_status(SERV_URL):
    print("[ERROR] Ollama 서버가 실행 중이지 않습니다.\n터미널에서 'ollama serve'를 실행하거나, 설치 여부를 확인하세요.")
else:
    print(f"{SERV_URL} IS RUNNING.")

# 2. Few-Shot Prompting Projects
## 2-1. 코드 리뷰 및 컨벤션 체크 (Code Reviewer)
#### 외부 서버로 코드를 보낼 수 없는 보안 환경에서 코드 리뷰 및 컨벤션 체크 (Code Reviewer)할 수 있는 로컬 LLM 프로젝트
#### 사내의 특정 코딩 스타일 가이드나 보안 규칙을 로컬 LLM에게 가르치는 프로젝트
#### 2-1-1. 규칙의 구체화 (Prefix): Prefix에 사내 위키(Wiki) 등에 정리된 코딩 스타일 가이드라인을 요약해서 입력
#### 2-1-2. Task에 맞는 모델 선택
###### - 성능 중심: Llama-3-8B
###### - 코드 특화: CodeLlama-7B, DeepSeek-Coder-6.7B
###### - 저사양 환경: Phi-3-mini
#### 2-1-3. CI/CD 연동: 이 프로젝트를 Git의 Pre-commit hook으로 등록해, 커밋 시 로컬 모델이 자동으로 코드 검사 후, 통과 못 하면 커밋 막는 기능 구현 해볼까?

In [ ]:
import requests
from langchain_community.llms import Ollama
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# 1. 로컬 LLM 설정 (코드 리뷰에 특화된 모델 검색 e.g CodeLlama, DeepSeek-Coder)
llm = Ollama(model="codellama", temperature=0)

# 2. 사내 컨벤션 및 보안 규칙 Few-shot 예시 정의
examples = [
    {
        "bad_code": "var user_name = 'admin';",
        "review": """- [컨벤션] 'var' 대신 'const' 또는 'let'을 사용하세요.
- [명명법] 변수명은 camelCase(userName)를 권장합니다.
- [보안] 하드코딩된 계정 정보가 있는지 확인이 필요합니다."""
    },
    {
        "bad_code": "if(a == 10) {{ return true; }}",  # # [20260112] 예시 데이터의 코드 내 중괄호를 {{ }}로 변경. 중괄호 에러 방지 (Brace Handling)
        "review": """- [컨벤션] 동등 연산자(==) 대신 일치 연산자(===)를 사용하여 타입 안전성을 확보하세요.
- [가독성] 변수명 'a'는 의미를 알기 어렵습니다. 역할을 알 수 있는 이름으로 수정하세요."""
    },
    {
        "bad_code": "const query = 'SELECT * FROM users WHERE id = ' + id;",
        "review": """- [보안] SQL Injection 위험이 있습니다. Parameterized Query 또는 ORM을 사용하세요.
- [보안] 유저 전체 정보(*) 대신 필요한 컬럼만 명시하세요."""
    }
]

# 3. 예시 포맷 정의
example_prompt = PromptTemplate(
    input_variables=["bad_code", "review"],
    template="[대상 코드]\n{bad_code}\n\n[리뷰 결과]\n{review}"
)

# 4. 전체 Few-shot 프롬프트 구성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""당신은 사내 코드 리뷰어입니다. 다음 규칙을 준수하여 입력된 코드를 검토하세요:
1. 변수명은 camelCase를 사용한다.
2. 'var' 사용을 금지한다.
3. 보안 취약점(SQL Injection, 하드코딩 등)을 최우선으로 지적한다.
4. 답변은 간결하게 리스트 형식으로 출력한다.""",
    suffix="\n[대상 코드]\n{input}\n\n[리뷰 결과]\n",
    input_variables=["input"],
    example_separator="\n\n---\n\n"
)

# 5. 코드 리뷰 실행
target_code = """
function save_data(input_val) {
    var query = "INSERT INTO logs VALUES ('" + input_val + "')";
    db.execute(query);
}
"""

final_prompt = few_shot_prompt.format(input=target_code)

print("로컬 LLM 코드 리뷰 GO!GO!GO!")
print("-" * 30)

try:
    response = llm.invoke(final_prompt)
    print(response)
except Exception as e:
    print(f"오류 발생: {e}\nOllama 서버가 실행 중인지 확인하세요.")

## 2-2. 정형 데이터 추출기 (Structured Data Extractor)
#### 2-2-1. Temperature 설정 (temperature=0): 정확한 답변 위해
#### 2-2-2. 품질의 예시 (Examples): Few-shot의 핵심은 "모델이 패턴을 인지하게 하는 것"
#### 2-2-3. Prefix와 Suffix의 역할
###### - Prefix: "반드시 JSON 포맷만 출력하세요"라는 제약 사항을 걸어 모델이 "네, 알겠습니다" 같은 불필요한 서두를 떼지 않게 함
###### - Suffix: 사용자의 실제 입력값(input_mail)을 배치하고, 마지막에 추출 결과(JSON):라는 문구를 두어 모델이 바로 다음 토큰으로 {를 생성하도록 유도
#### 2-2-4. 일부 데이터 누락된 예시 포함: (아는 척 못 하게) Hallucination 방지

In [ ]:
import json
from langchain_community.llms import Ollama
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# 1. 로컬 LLM 설정 (예: llama3, mistral, 혹은 한국어 특화 모델)
llm = Ollama(model="llama3", temperature=0) # TODO 추출 작업은 정밀도 중요 temperature=0 설정

# 2. Few-shot을 위한 예시 데이터 (Examples) 정의
# 모델에게 어떤 비정형 텍스트에서 어떤 JSON 결과가 나와야 하는지 학습 예시 [20260112] 예시 데이터의 JSON 중괄호를 {{ }}로 변경. 중괄호 에러 방지 (Brace Handling)
examples = [
    {
        "mail_text": "안녕하세요, 구글 코리아의 데이터 분석가 김철수입니다. 2023년 5월 10일에 입사하여 업무 보고 드립니다.",
        "answer": """{{"name": "김철수", "company": "구글 코리아", "role": "데이터 분석가", "date": "2023-05-10", "summary": "입사 후, 업무 보고"}}"""
    },
    {
        "mail_text": "네이버 마케팅팀 팀장 박영희입니다. 오는 2024년 1월 15일 미팅 일정 확인 부탁드립니다.",
        "answer": """{{"name": "박영희", "company": "네이버", "role": "마케팅팀 팀장", "date": "2024-01-15", "summary": "미팅 일정 확인"}}"""
    },
    {
        "mail_text": "삼성전자 폰팔이 이재용입니다. 프로젝트는 2023년 12월 25일에 종료될 예정입니다.",
        "answer": """{{"name": "이재용", "company": "삼성전자", "role": "폰팔이", "date": "2023-12-25", "summary": "프로젝트 종료 예정"}}"""
    },
    {
        "mail_text": "현대 중고차 딜러 정몽구입니다. 차보러 언제 오시나요?",  # # TODO [중요] 누락된 데이터(예: 날짜가 없는 경우)에 "정보 없음" 혹은 **null** 출력하는 예시 포함해 Hallucination 방지
        "answer": """{{"name": "정몽구", "company": "현대 중고차", "role": "중고차 딜러", "date": "정보 없음", "summary": "방문 일정"}}"""
    },
]

# 3. 예시를 보여줄 포맷(Template) 정의
example_formatter_template = """
메일 내용: {mail_text}
추출 결과(JSON): {answer}
"""
example_prompt = PromptTemplate(
    input_variables=["mail_text", "answer"],
    template=example_formatter_template,
)

# 4. Few-shot 프롬프트 구성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    # 지시사항 부분에 예시 형식을 적을 때도 {{ }} 사용. 중괄호 에러 방지 (Brace Handling)
    prefix="사내 메일에서 정보를 추출하여 {{\"name\": \"...\", \"company\": \"...\"}} 형식의 JSON으로 답하세요.",
    # prefix="다음 사내 메일 텍스트에서 이름, 회사, 역할, 날짜를 추출하여 JSON 형식으로 답변하세요. 반드시 JSON 포맷만 출력하세요.",
    suffix="메일 내용: {input}\n추출 결과(JSON):",
    input_variables=["input"],
    example_separator="\n\n",
)

# 5. 실행
input_mail = """
안녕하세요.

경영지원팀 이세돌 입니다.

엑셀 파일도 작성 후 회신 부탁 드립니다.

감사합니다.
"""
final_prompt = few_shot_prompt.format(input=input_mail)

print("--- [생성된 프롬프트] ---")
print(final_prompt)

print("\n--- [LLM 추출 결과] ---")
response = llm.invoke(final_prompt)
print(response)

# 6. JSON 파싱
try:
    data = json.loads(response)
    print("\n[파싱 성공]")
    print(f"이름: {data['name']}, 주요내용: {data['summary']}, 날짜: {data['date']}, 직책: {data['role']}, 회사: {data['company']}")
except Exception as e:
    print("\n[파싱 실패]", e)

## 2-3. 로컬 LLM 기반 데이터 분석가(Local Data Scientist) 구축
#### 민감한 로컬 데이터를 외부로 유출하지 않고도 정교한 분석 보고서를 작성
#### 2-3-1. 프로젝트 설계 구조
###### - 모델: Llama-3-8B(일반적), Mistral(분석 특화)
###### - 데이터 전략: 원본 데이터 전체를 프롬프트에 넣기보다는, **데이터의 샘플(Top 5 rows)**과 통계 요약 정보를 추출하여 프롬프트에 제공
###### - Few-shot 예시: '데이터 요약 → 가설 설정 → 분석 결과 → 비즈니스 제언'으로 이어지는 보고서 형식 학습



In [8]:
import json
from langchain_community.llms import Ollama   # # from langchain_ollama import OllamaLLM  # # TODO 차후 최신 OllamaLLM 로 변경 필요
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# 1. 로컬 LLM 설정  # # TODO Ollama의 mistral-7B, mistral-nemo, mistral-small 상업적 사용 가능 But!!! 3060(mistral-nemo(12B)) 이나 VRAM(3090/4090 mistral-small(22B)) 필요
llm = Ollama(model="mistral", temperature=0.1) # # 예시(Few-shot) 형식을 엄격히 따르고, 답변이 영어되는 현상 방지 위해 temperature를 0 또는 0.1로 낮춤
# llm = OllamaLLM(model="mistral", temperature=0)  # # TODO 차후 최신 OllamaLLM 로 변경 필요

# 2. 데이터 분석 Few-shot 예시 정의 (데이터 요약 -> 가설 -> 결과 -> 제언)
examples = [  # # TODO 분석하려는 도메인 카테고리 따라 예시 다르게 입력???
#     # 1. 이커머스: 반품률 분석
#     {
#         "raw_data": "신규 런칭 가전 브랜드: 판매량 5,000대 돌파, 반품률 15% (평균 대비 3배), 주요 사유 '설치 어려움'",
#         "analysis_report": """
# 1. 데이터 요약: 판매량은 높으나 초기 반품률이 비정상적으로 높아 수익성이 악화됨.
# 2. 가설 설정: 제품 자체의 결함보다는 사용자 매뉴얼의 가독성이 낮아 설치에 실패했을 것이다.
# 3. 분석 결과: 고객센터 문의 데이터 분석 결과, 설치 단계 중 '배선 연결' 질문이 전체의 70%를 차지함.
# 4. 비즈니스 제언: QR코드를 활용한 '30초 설치 가이드 영상'을 제품 박스 상단에 부착하고, 자가 설치 지원 챗봇을 강화해야 함.
# """
#     },
#     # 2. 인사 관리: 직원 퇴사율
#     {
#         "raw_data": "IT 기업 개발팀 퇴사율: 입사 1년 미만 신입 사원 퇴사율 35%, 면담 사유 '성장 기회 부족' 다수",
#         "analysis_report": """
# 1. 데이터 요약: 신입 개발자의 조기 이탈 현상이 심화되어 채용 비용 손실이 발생함.
# 2. 가설 설정: 실무 투입 전 체계적인 온보딩 및 멘토링 프로그램이 부재하여 소외감을 느꼈을 것이다.
# 3. 분석 결과: 멘토링이 제공된 팀의 퇴사율은 5%인 반면, 방치된 팀은 45%로 극명한 차이를 보임.
# 4. 비즈니스 제언: '신입 사원 1:1 기술 멘토링' 제도를 의무화하고, 분기별 기술 세미나 발표 기회를 제공하여 성장 경험을 부여해야 함.
# """
#     },
#     # 3. 마케팅: 광고 채널 효율
#     {
#         "raw_data": "SNS 광고 성과: 인스타그램 도달률 200% 증가, 전환율(CVR) 0.5% (하락), 클릭당 비용(CPC) 상승",
#         "analysis_report": """
# 1. 데이터 요약: 광고 노출은 크게 늘었으나 실제 구매로 이어지는 비율은 오히려 낮아짐.
# 2. 가설 설정: 광고 타겟팅 범위가 너무 넓어져서 구매 의사가 낮은 대중에게까지 노출되었을 것이다.
# 3. 분석 결과: 클릭 고객의 페이지 체류 시간이 평균 5초 미만으로, 랜딩 페이지의 소구점이 고객 기대와 일치하지 않음.
# 4. 비즈니스 제언: 타겟 범위를 '관심사' 기반에서 '유사 구매 이력' 기반으로 좁히고, 광고 소재와 랜딩 페이지의 메시지를 일치시키는 리뉴얼 필요.
# """
#     },
#     # 4. 제조: 공정 불량률
#     {
#         "raw_data": "반도체 공정 라인: 야간 교대 근무 시 불량률 12% (주간은 4%), 특정 장비 온도 센서 과열 기록",
#         "analysis_report": """
# 1. 데이터 요약: 주야간 불량률 격차가 크며, 야간 시간대 장비 과열 문제가 발생함.
# 2. 가설 설정: 야간 조의 냉각 시스템 관리 미숙 또는 냉각수 교체 주기 지연이 원인일 것이다.
# 3. 분석 결과: 로그 데이터 확인 결과, 야간 시간대 냉각수 펌프 가동 압력이 기준치보다 낮게 설정되어 있음.
# 4. 비즈니스 제언: 냉각 시스템 가동을 자동화 설정으로 고정하고, 이상 온도 감지 시 즉시 야간 관리자에게 모바일 알림이 가는 시스템을 구축해야 함.
# """
#     },
#     # 5. 금융: 카드 소비 패턴
#     {
#         "raw_data": "2030 세대 카드 이용: 해외 여행 결제액 50% 증가, 해외 직구 결제액 10% 감소, 환전 수수료 불만 민원 증가",
#         "analysis_report": """
# 1. 데이터 요약: 직구보다는 직접 여행을 떠나는 수요가 폭증했으며, 환전 비용에 민감한 반응을 보임.
# 2. 가설 설정: 고물가로 인해 배송비가 비싼 직구 대신 직접 쇼핑을 선호하며, 실속형 여행자가 늘었을 것이다.
# 3. 분석 결과: 수수료 면제 혜택이 있는 타사 '트래블 카드'로의 고객 이탈 수치가 전월 대비 15% 상승함.
# 4. 비즈니스 제언: 해외 결제 수수료 면제 및 현지 ATM 출금 무료 혜택을 담은 신규 특화 카드 상품 런칭이 시급함.
# """
#     },
#     # 6. 콘텐츠 플랫폼: 구독 유지율
#     {
#         "raw_data": "OTT 서비스: 신규 가입자 20% 증가, 한 달 뒤 해지율 60%, 인기 예능 'A' 종료 시점과 일치",
#         "analysis_report": """
# 1. 데이터 요약: 특정 킬러 콘텐츠 의존도가 너무 높아 콘텐츠 종료 후 대거 이탈 발생.
# 2. 가설 설정: 'A' 예능 시청자들에게 후속으로 볼만한 유사 장르의 추천이 이루어지지 않았을 것이다.
# 3. 분석 결과: 해지 고객의 80%가 'A' 외에 다른 콘텐츠를 2개 이하로 시청함.
# 4. 비즈니스 제언: 인기 콘텐츠 종료 전 유사 감성의 시리즈를 선공개하고, 개인화 추천 알고리즘에 '장르 확장성' 가중치를 부여해야 함.
# """
#     },
#     # 7. 식음료(F&B): 신메뉴 반응
#     {
#         "raw_data": "카페 프랜차이즈: 시즌 음료 '민트초코라떼' 판매 비중 2%, SNS 언급량은 전체 1위, 재구매율 0.5%",
#         "analysis_report": """
# 1. 데이터 요약: 화제성은 매우 높으나 실제 판매와 재구매로는 연결되지 않는 '바이럴 전용' 메뉴임.
# 2. 가설 설정: 비주얼은 사진 찍기에 좋으나 맛이 대중적이지 않아 일회성 소비에 그쳤을 것이다.
# 3. 분석 결과: POS 데이터 기준, 단품 주문은 많으나 베이커리류와의 세트 구매가 거의 없음(맛의 조화 실패).
# 4. 비즈니스 제언: 한정 판매로 화제성만 챙기고 조기 종료하거나, 맛을 대중적으로 개선하여 '시즌 2'로 재출시하여 매니아층을 확보해야 함.
# """
#     },
#     # 8. 소매업: 매장 진열 효율
#     {
#         "raw_data": "대형마트 A지점: 주류 코너 매출 5% 상승, 안주류(스낵) 매출 30% 상승, 동선 분석상 두 코너 인접 배치 후 변화",
#         "analysis_report": """
# 1. 데이터 요약: 주류와 안주류의 연관 구매가 활성화되어 객단가가 상승함.
# 2. 가설 설정: 물리적 거리가 가까워짐에 따라 '번들 구매'에 대한 인지적 노력이 줄어들었을 것이다.
# 3. 분석 결과: 장바구니 분석 결과 주류 구매자의 75%가 인접한 스낵류를 동시에 구매함.
# 4. 비즈니스 제언: 단순 인접 배치를 넘어 '와인+치즈', '맥주+나초' 등 구체적인 큐레이션 매대를 구성하여 교차 판매를 더욱 극대화해야 함.
# """
#     },
#     # 9. 교육 서비스: 학습 완강률
#     {
#         "raw_data": "온라인 강의 플랫폼: 무료 강좌 완강률 5%, 유료 강좌(5만원 이상) 완강률 45%, 챕터 3에서 이탈률 최고조",
#         "analysis_report": """
# 1. 데이터 요약: 지불 금액에 따라 학습 의지가 크게 다르며, 강의 초반 특정 구간에 고비가 존재함.
# 2. 가설 설정: 챕터 3의 난이도가 갑자기 높아지거나 실습 환경 구축이 어려워 포기자가 발생할 것이다.
# 3. 분석 결과: 챕터 3의 Q&A 게시글 수가 타 챕터 대비 4배 많으며 대부분 '에러 해결' 관련임.
# 4. 비즈니스 제언: 챕터 3 시작 전 '트러블슈팅 가이드'를 강화하고, 완강 시 일정 금액을 환급해주는 '챌린지형 환급반' 모델을 도입하여 동기를 부여해야 함.
# """
#     },
#     # 10. 헬스케어: 앱 리텐션
#     {
#         "raw_data": "운동 기록 앱: 일일 활성 사용자(DAU) 평일 대비 주말 40% 하락, 주말 알림 클릭률 2%",
#         "analysis_report": """
# 1. 데이터 요약: 주말에는 사용자의 운동 의지가 급격히 꺾이며 기존 알림이 효과를 발휘하지 못함.
# 2. 가설 설정: 평일과 동일한 '강도 높은 운동' 독려 알림이 주말의 휴식 심리와 충돌하여 거부감을 주었을 것이다.
# 3. 분석 결과: 주말에 '가벼운 산책'이나 '스트레칭' 제안 알림을 보낸 실험군은 리텐션이 15% 높게 나타남.
# 4. 비즈니스 제언: 주말 전용 '휴식 및 리프레시' 컨셉의 콘텐츠를 배포하고 알림 문구를 부드러운 톤으로 변경하는 개인화 전략 필요.
# """
#     },
    # 11. 쇼핑몰: 매출
    {
        "raw_data": "A쇼핑몰 8월 매출: 전월 대비 여성 의류 30% 증가, 남성 의류 5% 감소, 장바구니 포기율 40%",
        "analysis_report": """
1. 데이터 요약: 8월 매출은 여성 의류가 성장을 주도했으나 남성 부문은 정체됨.
2. 가설 설정: 시즌 종료 세일이 여성 고객에게만 더 강력한 소구력을 가졌을 것이다.
3. 분석 결과: 데이터 확인 결과 여성 고객의 재방문율이 남성보다 2.5배 높음.
4. 비즈니스 제언: 9월 신상품 런칭 시 남성 고객 전용 '첫 구매 할인 쿠폰'을 발행하여 이탈을 방지해야 함.
"""
    },
    # 12. 정보시스템 관리: 서버 가용량
    {
        "raw_data": "클라우드 서버 가용성: 장애 시간 2시간 발생, CPU 점유율 90% 지속, 특정 API 호출 급증",
        "analysis_report": """
1. 데이터 요약: 특정 시간대 서버 부하로 인한 가용성 저하 발생.
2. 가설 설정: 신규 런칭한 모바일 앱의 푸시 알림이 API 서버에 동시 접속 부하를 유도했을 것이다.
3. 분석 결과: 푸시 발송 시점과 API 호출 급증 시점이 99% 일치함이 확인됨.
4. 비즈니스 제언: 푸시 알림 발송 시 '배치(Batch)' 방식으로 시간차 분산 발송 로직을 도입해야 함.
"""
    }
]

# 3. 예시 포맷 정의
example_prompt = PromptTemplate(
    input_variables=["raw_data", "analysis_report"],
    template="[입력 데이터]\n{raw_data}\n\n[분석 결과 보고서]\n{analysis_report}"
)

# 4. 전체 Few-shot 프롬프트 구성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""당신은 데이터 과학자입니다. 제공된 데이터를 분석하여 보고서를 작성하세요.
반드시 아래 4단계 구조를 엄격히 지켜 한국어로만 작성하세요:
1. 데이터 요약
2. 가설 설정
3. 분석 결과
4. 비즈니스 제언""",  # # 영어 답변 방지 위해, prefix에 언어 한국어 제한
    suffix="\n[입력 데이터]\n{input}\n\n[분석 결과 보고서]\n1. 데이터 요약:",  # # LLM 답변의 사족을 줄이고, LLM 모델이 이어지는 텍스트를 생성하는 성질 이용해, 보고서 시작 부분 추가 "1. 데이터 요약:"
    input_variables=["input"],
    example_separator="\n\n---\n\n"
)

# 5. 실제 데이터 분석 실행
# # TODO 실 서비스 시, 데이터 구조 파악용 샘플 요약 df.head() 후, 분석 보고서, 그리고 pandas.describe(), df.corr() 등의 통계값 추가하는 방향??? matplotlib 차트도 추가???
target_data = "배달 앱 10월 로그: 야간 시간대(22시~02시) 주문 15% 하락, 배달비 평균 2,000원 상승, 라이더 수 10% 감소"

final_prompt = few_shot_prompt.format(input=target_data)

print("로컬 데이터 분석가가 보고서를 생성 중...")
print("-" * 30)

try:
    # 6. 실행
    response = llm.invoke(final_prompt)
    # Suffix에서 시작한 문구를 합쳐서 출력
    print(f"1. 데이터 요약:{response}\n\n")
except Exception as e:
    print(f"연결 오류: {e}. Ollama 서버가 켜져 있는지 확인하세요.\n")

로컬 데이터 분석가가 보고서를 생성 중...
------------------------------
1. 데이터 요약: 10월 로그에서는 야간 시간대(22시~02시)의 주문 수가 15% 감소, 배달비 평균이 2,000원 상승, 라이더 수도 10% 감소가 확인되었습니다.

2. 가설 설정:
- 야간 시간대에서의 주문 하락은 고객의 요구사항 변화로 인한 것일 수 있습니다.
- 배달비 상승과 라이더 수 감소는 라이더 부족으로 인한 추가적인 배달비 요금 조정이 있었을 수 있습니다.

3. 분석 결과:
- 고객 설문조사를 통해 야간 시간대에서의 주문 하락은 배달비 상승으로 인한 고객 불만이 있다는 피드백을 받았습니다.
- 라이더 부족으로 인한 추가적인 배달비 요금 조정은 확인되지 않았습니다.

4. 비즈니스 제언:
- 고객의 불만을 해소하기 위해서는 야간 시간대에서의 배달비를 조정하여 고객 요구사항에 맞게 가격을 절감할 필요가 있습니다.
- 라이더 부족 문제는 추후 개선 방안을 수립하여 해결해야 합니다.




In [9]:
target_input_list = ["A쇼핑몰: 여성 패션 매출 +20%, 남성 패션 -5%, 20대 유입량 급증.", "반도체 제조 라인: 수율 95% 유지 중, 특정 세정 장비 온도 2도 상승, 불량 감지 0.5% 증가.", "신규 앱 서비스: 주간 활성 사용자(WAU) 15% 상승, 결제 전환율 2% 정체, 결제 페이지 이탈률 70%.", "배달 서비스: 주말 주문량 +15%, 라이더 배차 시간 +10분, 고객 불만 건수 2배 증가."]

for target_input in target_input_list:
    final_prompt = few_shot_prompt.format(input=target_input)

    print(f"\ntarget_input: {target_input}")
    print("-" * 30)
    try:
        # 6. 실행
        response = llm.invoke(final_prompt)
        # Suffix에서 시작한 문구를 합쳐서 출력
        print(f"1. 데이터 요약:{response}\n\n\n")
    except Exception as e:
        print(f"연결 오류: {e}. Ollama 서버가 켜져 있는지 확인하세요.\n")
""" temperature=0
target_input: A쇼핑몰: 여성 패션 매출 +20%, 남성 패션 -5%, 20대 유입량 급증.
------------------------------
1. 데이터 요약: 20대 유입량의 급증으로 인해 여성 패션 매출이 20% 상승했지만, 남성 패션은 -5% 감소한 것으로 나타났습니다.

2. 가설 설정:
    - 20대 유입량의 급증이 여성 고객에게만 더 강력한 소구력을 가졌을 것이다.
    - 20대 유입량의 급증은 남성 패션 매출의 감소를 주도했을 것이다.

3. 분석 결과:
    - 데이터 확인 결과, 20대 유입량이 여성 고객에게만 더 강력한 소구력을 가졌음이 확인되었습니다.
    - 또한, 남성 패션 매출의 감소는 20대 유입량의 급증으로 주도된 것으로 나타났습니다.

4. 비즈니스 제언:
    - 9월 신상품 런칭 시, 남성 고객에게 더 강력한 소구력을 주기 위해 20대 유입량 유치 활동을 강화하는 것이 필요합니다.
    - 또한, 남성 패션 매출의 감소를 보완하기 위해서는 남성 고객에게 더 맞춤화된 마케팅 전략을 도입하는 것이 필요합니다.




target_input: 반도체 제조 라인: 수율 95% 유지 중, 특정 세정 장비 온도 2도 상승, 불량 감지 0.5% 증가.
------------------------------
1. 데이터 요약: 특정 세정 장비의 온도 상승으로 인한 불량 제품 생성 증가.
2. 가설 설정: 세정 장비의 온도 조절 문제로 유발된 현상이며, 이를 해결하기 위해서는 온도 조절 시스템을 검사하고 필요에 따라 수리가 필요할 것이다.
3. 분석 결과: 데이터 분석 결과 특정 세정 장비의 온도 상승으로 인한 불량 제품 생성 증가에 대응하기 위해서는 온도 조절 시스템을 검사하고 필요에 따라 수리가 필요함이 확인됨.
4. 비즈니스 제언: 세정 장비의 온도 조절 문제를 해결하기 위해서는 온도 조절 시스템을 검사하고 필요에 따라 수리가 필요함으로, 이를 지속적으로 모니터링하고 필요한 대응을 취해야 함.




target_input: 신규 앱 서비스: 주간 활성 사용자(WAU) 15% 상승, 결제 전환율 2% 정체, 결제 페이지 이탈률 70%.
------------------------------
1. 데이터 요약: 신규 앱 서비스에서는 주간 활성 사용자(WAU)가 15% 상승하였으나, 결제 전환율은 2% 정체되었고 결제 페이지 이탈률이 70%로 매우 높아졌습니다.

2. 가설 설정:
- 신규 사용자의 이해도와 사용 경험에 대한 불만족감이 결제 페이지 이탈을 유발했을 것입니다.
- 결제 전환율이 정체된 것은 신규 사용자의 구매 의도가 있음에도 불구하고 결제 페이지 이탈로 인한 문제입니다.

3. 분석 결과:
- 사용자 설문조사를 통해 신규 사용자의 이해도와 사용 경험에 대한 불만족감을 확인할 수 있었습니다.
- 결제 페이지 이탈 분석을 통해 결제 페이지의 UX/UI 문제를 파악할 수 있었습니다.

4. 비즈니스 제언:
- 신규 사용자에게 더 나은 이해도와 사용 경험을 제공하기 위한 교육 및 가이드라인 작성이 필요합니다.
- 결제 페이지의 UX/UI를 개선하여 이탈률을 줄이고 결제 전환율을 증대시키는 데 노력해야 합니다.




target_input: 배달 서비스: 주말 주문량 +15%, 라이더 배차 시간 +10분, 고객 불만 건수 2배 증가.
------------------------------
1. 데이터 요약: 주말에는 주문량이 증가하였으나 라이더 배차 시간과 고객 불만 건수가 상승했습니다.

2. 가설 설정:
주말에 많은 고객들이 배달 서비스를 사용하여 라이더 부족으로 인한 배차 시간 증가와 고객 불만의 상승을 유발했을 것입니다.

3. 분석 결과:
데이터 분석 결과, 주말에 라이더 부족으로 인한 배차 시간 증가와 고객 불만의 상승은 일치하는 추세를 보였습니다.

4. 비즈니스 제언:
라이더 부족으로 인한 배차 시간 증가와 고객 불만의 상승을 해소하기 위해서는, 주말에도 적절한 수량의 라이더를 배치하여 배달 서비스의 품질을 유지하고 고객 불만을 줄이는 것이 필요합니다.
"""
# # TODO :::::::::::::::::::::::::::::::::::::::::::::::
"""temperature=0.1
target_input: A쇼핑몰: 여성 패션 매출 +20%, 남성 패션 -5%, 20대 유입량 급증.
------------------------------
1. 데이터 요약: 20대 유입량의 급증으로 인해 여성 패션 매출이 20% 상승하였으나, 남성 패션은 -5% 감소한 것으로 확인됨.

2. 가설 설정:
    - 20대 유입량의 급증이 여성 고객에게만 더 강력한 소구력을 가졌을 것이다.
    - 20대 유입량의 급증은 남성 패션 매출의 감소를 주도했을 것이다.

3. 분석 결과:
    - 여성 고객의 재방문율이 남성보다 2.5배 높음으로, 가설1은 확인됨.
    - 20대 유입량의 급증이 남성 패션 매출의 감소를 주도했을 것으로, 가설2는 확인되지 않음.

4. 비즈니스 제언:
    - 9월 신상품 런칭 시 남성 고객 전용 '첫 구매 할인 쿠폰'을 발행하여 이탈을 방지해야 함. (이미 확인된 결과에서 유도됨)
    - 남성 패션 매출의 감소를 해소하기 위한 추가 전략이 필요할 것으로 예상됨. (가설2가 확인되지 않음에 따른 추측)




target_input: 반도체 제조 라인: 수율 95% 유지 중, 특정 세정 장비 온도 2도 상승, 불량 감지 0.5% 증가.
------------------------------
1. 데이터 요약: 특정 세정 장비의 온도 상승으로 인한 불량 발생 증가 추측.
2. 가설 설정: 세정 장비 온도 조정에 따른 불량 감지 율의 변동이 있을 것으로 예상됨.
3. 분석 결과: 데이터 분석 결과 2도 온도 상승 시 0.5% 증가된 불량 발생 확인됨.
4. 비즈니스 제언: 세정 장비의 온도를 조정하여 불량 감지 율을 줄이고, 품질 보증을 위한 추가적인 검사 및 제어 절차 수립이 필요함.




target_input: 신규 앱 서비스: 주간 활성 사용자(WAU) 15% 상승, 결제 전환율 2% 정체, 결제 페이지 이탈률 70%.
------------------------------
1. 데이터 요약: 신규 앱 서비스에서는 주간 활성 사용자(WAU)가 15% 상승하였으나, 결제 전환율은 2% 정체되었고 결제 페이지 이탈률이 70%로 매우 높아졌습니다.

2. 가설 설정:
- 신규 사용자의 이해도와 사용 경험에 대한 문제점이 있어 결제 전환율이 저하되었을 것입니다.
- 결제 페이지 설계나 UX/UI 등의 문제로 인해 사용자가 결제 페이지에서 이탈하고 있을 수 있습니다.

3. 분석 결과:
- 신규 사용자와 기존 사용자의 이해도와 사용 경험을 분석하여 문제점을 파악합니다.
- A/B 테스팅을 통해 결제 페이지의 UX/UI를 개선하고, 신규 사용자에게 가이드나 튜토리얼을 제공하여 이해도와 사용 경험을 향상시키는 것이 필요합니다.

4. 비즈니스 제언:
- 신규 사용자의 이해도와 사용 경험을 개선하여 결제 전환율을 높이고, 결제 페이지 이탈률을 줄이는 것이 중요합니다.
- 신규 사용자에게 가이드나 튜토리얼을 제공하여 이해도와 사용 경험을 향상시키고, A/B 테스팅을 통해 결제 페이지의 UX/UI를 개선하는 것이 필요합니다.




target_input: 배달 서비스: 주말 주문량 +15%, 라이더 배차 시간 +10분, 고객 불만 건수 2배 증가.
------------------------------
1. 데이터 요약: 주말에는 주문량이 증가하였으나 라이더 배차 시간과 고객 불만 건수가 증가함을 확인할 수 있습니다.

2. 가설 설정:
- 주말에는 고객의 요구사항이 높아져서 라이더 배차 시간이 길어지고 불만 건수가 증가했을 것입니다.
- 주말에는 휴대폰 사용자의 활동량이 높아져서 배달 서비스를 더 많이 이용하여 주문량이 증가했을 것입니다.

3. 분석 결과:
- 데이터 분석 결과, 주말에는 고객의 요구사항이 높아져서 라이더 배차 시간이 길어지고 불만 건수가 증가했습니다.
- 또한, 주말에는 휴대폰 사용자의 활동량이 높아져서 배달 서비스를 더 많이 이용하여 주문량이 증가했습니다.

4. 비즈니스 제언:
- 고객의 요구사항을 더 잘 만족하기 위해서는 라이더 배차 시간을 줄이고 고객 불만에 대한 대처방안을 개선해야 합니다.
- 또한, 주말에는 휴대폰 사용자의 활동량이 높아져서 배달 서비스를 더 많이 이용하고 있으므로, 서비스 품질을 유지하기 위해서는 추가적인 라이더를 채용하거나 라이더의 수익을 보장하는 방안을 고려해야 합니다.
"""


target_input: A쇼핑몰: 여성 패션 매출 +20%, 남성 패션 -5%, 20대 유입량 급증.
------------------------------
1. 데이터 요약: 20대 유입량의 급증으로 인해 여성 패션 매출이 20% 상승하였으나, 남성 패션은 -5% 감소한 것으로 확인됨.

2. 가설 설정:
    - 20대 유입량의 급증이 여성 고객에게만 더 강력한 소구력을 가졌을 것이다.
    - 20대 유입량의 급증은 남성 패션 매출의 감소를 주도했을 것이다.

3. 분석 결과:
    - 여성 고객의 재방문율이 남성보다 2.5배 높음으로, 가설1은 확인됨.
    - 20대 유입량의 급증이 남성 패션 매출의 감소를 주도했을 것으로, 가설2는 확인되지 않음.

4. 비즈니스 제언:
    - 9월 신상품 런칭 시 남성 고객 전용 '첫 구매 할인 쿠폰'을 발행하여 이탈을 방지해야 함. (이미 확인된 결과에서 유도됨)
    - 남성 패션 매출의 감소를 해소하기 위한 추가 전략이 필요할 것으로 예상됨. (가설2가 확인되지 않음에 따른 추측)




target_input: 반도체 제조 라인: 수율 95% 유지 중, 특정 세정 장비 온도 2도 상승, 불량 감지 0.5% 증가.
------------------------------
1. 데이터 요약: 특정 세정 장비의 온도 상승으로 인한 불량 발생 증가 추측.
2. 가설 설정: 세정 장비 온도 조정에 따른 불량 감지 율의 변동이 있을 것으로 예상됨.
3. 분석 결과: 데이터 분석 결과 2도 온도 상승 시 0.5% 증가된 불량 발생 확인됨.
4. 비즈니스 제언: 세정 장비의 온도를 조정하여 불량 감지 율을 줄이고, 품질 보증을 위한 추가적인 검사 및 제어 절차 수립이 필요함.




target_input: 신규 앱 서비스: 주간 활성 사용자(WAU) 15% 상승, 결제 전환율 2% 정체, 결제 페이지 이탈률 70%.
------------------------------
1. 데이터 요약: 신규 앱 서비스

'\ntarget_input: A쇼핑몰: 여성 패션 매출 +20%, 남성 패션 -5%, 20대 유입량 급증.\n------------------------------\n1. 데이터 요약: 20대 유입량의 급증으로 인해 여성 패션 매출이 20% 상승했지만, 남성 패션은 -5% 감소한 것으로 나타났습니다.\n\n2. 가설 설정:\n    - 20대 유입량의 급증이 여성 고객에게만 더 강력한 소구력을 가졌을 것이다.\n    - 20대 유입량의 급증은 남성 패션 매출의 감소를 주도했을 것이다.\n\n3. 분석 결과:\n    - 데이터 확인 결과, 20대 유입량이 여성 고객에게만 더 강력한 소구력을 가졌음이 확인되었습니다.\n    - 또한, 남성 패션 매출의 감소는 20대 유입량의 급증으로 주도된 것으로 나타났습니다.\n\n4. 비즈니스 제언:\n    - 9월 신상품 런칭 시, 남성 고객에게 더 강력한 소구력을 주기 위해 20대 유입량 유치 활동을 강화하는 것이 필요합니다.\n    - 또한, 남성 패션 매출의 감소를 보완하기 위해서는 남성 고객에게 더 맞춤화된 마케팅 전략을 도입하는 것이 필요합니다.\n\n\n\n\ntarget_input: 반도체 제조 라인: 수율 95% 유지 중, 특정 세정 장비 온도 2도 상승, 불량 감지 0.5% 증가.\n------------------------------\n1. 데이터 요약: 특정 세정 장비의 온도 상승으로 인한 불량 제품 생성 증가.\n2. 가설 설정: 세정 장비의 온도 조절 문제로 유발된 현상이며, 이를 해결하기 위해서는 온도 조절 시스템을 검사하고 필요에 따라 수리가 필요할 것이다.\n3. 분석 결과: 데이터 분석 결과 특정 세정 장비의 온도 상승으로 인한 불량 제품 생성 증가에 대응하기 위해서는 온도 조절 시스템을 검사하고 필요에 따라 수리가 필요함이 확인됨.\n4. 비즈니스 제언: 세정 장비의 온도 조절 문제를 해결하기 위해서는 온도 조절 시스템을 검사하고 필요에 따라 수리가 필요함으로, 이를 지속적으로 모